In [1]:
import ktrain
from ktrain import text as txt
import pandas as pd

In [2]:
train = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/train_same_size_as_bal.csv")
test = pd.read_csv("s3://yelp-dataset-pt-9/spencer/data/sentiment/en/test_small.csv")

In [3]:
x_train, y_train = train['text'].tolist(), train['stars'].tolist()

In [4]:
x_test, y_test = test['text'].tolist(), test['pos_neg_3_is_pos'].tolist()

In [5]:
trn, val, preproc = txt.texts_from_array(x_train=x_train, y_train=y_train,
                                          x_test=x_test, y_test=y_test,
                                          preprocess_mode='distilbert',
                                          maxlen=256)

task: text regression (supply class_names argument if this is supposed to be classification task)
preprocessing train...
language: en
train sequence lengths:
	mean : 109
	95percentile : 305
	99percentile : 519


/Users/spenceradams/.local/share/virtualenvs/Yelp-ds-xpajUQwb/lib/python3.7/site-packages/ktrain/text/preprocessor.py:508: UserWarning: Task is being treated as TEXT REGRESSION because class_names argument was not supplied. If this is incorrect, supply class_names argument.
  'class_names argument was not supplied. ' + \


preprocessing test...
language: en
test sequence lengths:
	mean : 109
	95percentile : 305
	99percentile : 522


In [6]:
import pickle

pickle.dump( trn, open( "trn.p", "wb" ), protocol=4 )
pickle.dump( val, open( "val.p", "wb" ), protocol=4 )
pickle.dump( preproc, open( "preproc.p", "wb" ), protocol=4 )

In [7]:
import boto3
from botocore.exceptions import ClientError

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [8]:
bucket = "yelp-dataset-pt-9"

In [9]:
upload_file('trn.p', bucket, 'spencer/data/sentiment/en/ktrain/regression/trn.p')

True

In [10]:
upload_file('val.p', bucket, 'spencer/data/sentiment/en/ktrain/regression/val.p')

True

In [11]:
upload_file('preproc.p', bucket, 'spencer/data/sentiment/en/ktrain/regression/preproc.p')

True